# Imports

In [ ]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Param,
    Constraint,
    SolverFactory,
    TransformationFactory,
    Suffix,
    value,
)
from pyomo.dae import ContinuousSet, DerivativeVar
import idaes.core.solvers.get_solver

from pyomo.contrib.doe import DesignOfExperiments

from pyomo.contrib.parmest.experiment import Experiment
import pyomo.contrib.parmest.parmest as parmest
import numpy as np
import pandas as pd

# Experimental Data

In [2]:
time_points = np.linspace(0, 10, 31)
x1_values = np.array(
    [
        0,
        -0.049999932500001114,
        -0.11750006862498093,
        -0.18612499999997273,
        -0.2483936968183999,
        -0.3015753562741202,
        -0.3453494843749556,
        -0.3805128855089556,
        -0.40828518662989,
        -0.42995416172066814,
        -0.4467097812214418,
        -0.45957885908105717,
        -0.46941203914389995,
        -0.47689576500462116,
        -0.48257387710467886,
        -0.48687163148211804,
        -0.4901184572885179,
        -0.49256768832438497,
        -0.4944130698646676,
        -0.4958021843568668,
        -0.4968470696387529,
        -0.4976325595211166,
        -0.4982227739262201,
        -0.4986660939472598,
        -0.49899897791311437,
        -0.49924887786866806,
        -0.499436445455777,
        -0.499577206049966,
        -0.49968282769644984,
        -0.4997620748341597,
        -0.4998215284467895,
    ]
)

x2_values = np.array(
    [
        0,
        -0.19999987999995197,
        -0.3200000719999136,
        -0.39199999999994817,
        -0.4351999740799586,
        -0.46112001555196264,
        -0.47667199999997756,
        -0.4860031944012643,
        -0.49160192335921993,
        -0.49496115199999274,
        -0.4969766899906716,
        -0.49818601544559066,
        -0.4989116088319979,
        -0.49934696503798476,
        -0.4996081793362474,
        -0.4997649075077114,
        -0.4998589444482046,
        -0.4999153667366294,
        -0.49994922002166564,
        -0.4999695320008122,
        -0.4999817192151119,
        -0.4999890315246798,
        -0.4999934189121754,
        -0.49999605135046415,
        -0.49999763080933085,
        -0.49999857848502993,
        -0.4999991470917003,
        -0.4999994882548155,
        -0.4999996929527665,
        -0.49999981577180724,
        -0.49999988946304014,
    ]
)

u = np.full_like(time_points, -1)

data = pd.DataFrame({"t": time_points, "x1": x1_values, "x2": x2_values, "u": u})
data.head()

,t,x1,x2,u
0,0.000000,0.000000,0.0000,-1.0
1,0.333333,-0.050000,-0.2000,-1.0
2,0.666667,-0.117500,-0.3200,-1.0
3,1.000000,-0.186125,-0.3920,-1.0
4,1.333333,-0.248394,-0.4352,-1.0


# Experiment Class

In [3]:
class LinearDynamicalSystem(Experiment):
    def __init__(self, data, theta_init=None):
        """
        Args:
            data: DataFrame
                A DataFrame containing the data to be used for the experiment.
                The DataFrame should have columns 't', 'x1', 'x2', and 'u'.
            theta_init: dict, optional
                A dictionary containing the initial values of the parameters.
                The keys should be 'a11', 'a22', and 'b1'.
                If not provided, default values will be used.
        """
        self.data = data
        if theta_init is None:
            self.theta_init = {"a11": -1.0, "a22": -2.0, "b1": 0.0}
        else:
            self.theta_init = theta_init
        self.model = None

    def create_model(self):
        """
        In this method, we create the model, and define the variables, the parameters, and the constraints.
        """
        self.model = ConcreteModel()

        # Define time
        self.model.t = ContinuousSet(bounds=[0, 10])

        # Define variables
        self.model.x1 = Var(self.model.t, bounds=(-30, 30), initialize=0)
        self.model.x2 = Var(self.model.t, bounds=(-30, 30), initialize=0)
        self.model.u = Var(self.model.t, bounds=(-2, 2), initialize=0)

        # Define derivatives
        self.model.dx1dt = DerivativeVar(self.model.x1, wrt=self.model.t)
        self.model.dx2dt = DerivativeVar(self.model.x2, wrt=self.model.t)

        # Define parameters
        self.model.a11 = Var(initialize=self.theta_init["a11"])
        self.model.a12 = Var(initialize=1.0)
        self.model.a21 = Var(initialize=0.0)
        self.model.a22 = Var(initialize=self.theta_init["a22"])
        self.model.b1 = Var(initialize=self.theta_init["b1"])
        self.model.b2 = Var(initialize=1)

        # Define differential equations
        """
        def _ode1(m, t):
            return m.dx1dt[t] == m.a11 * m.x1[t] + m.a12*m.x2[t] + m.b1 * m.u[t]
        self.model.ode1 = Constraint(self.model.t, rule=_ode1)
        """

        @self.model.Constraint(self.model.t)
        def ode1(m, t):
            return m.dx1dt[t] == m.a11 * m.x1[t] + m.a12 * m.x2[t] + m.b1 * m.u[t]

        """
        def _ode2(m, t):
            return m.dx2dt[t] == m.a21 * m.x1[t] + m.a22*m.x2[t] + m.b2 * m.u[t]
        self.model.ode2 = Constraint(self.model.t, rule=_ode2)
        """

        @self.model.Constraint(self.model.t)
        def ode2(m, t):
            return m.dx2dt[t] == m.a21 * m.x1[t] + m.a22 * m.x2[t] + m.b2 * m.u[t]

        return None

    def finalize_model(self):
        """
        In this method, we finalize the model by fixing the initial conditions, the parameter values,
        and the input values. We also discretize the model if we have differential equations or if the data is continuous.
        """

        m = self.model

        # Update time discretization
        time = self.data["t"].to_numpy()
        # m.t.update((min(time), max(time)))
        m.t.update(time)
        m.t.set_changed(True)

        # Set initial points
        m.x1[0].fix(self.data["x1"].iloc[0])
        m.x2[0].fix(self.data["x2"].iloc[0])

        # Fix parameter values
        m.a11.fix()
        m.a12.fix()
        m.a21.fix()
        m.a22.fix()
        m.b1.fix()
        m.b2.fix()

        m.var_input = Suffix(direction=Suffix.LOCAL)
        m.var_input[m.u] = {0: -1}

        """
        sim = Simulator(m, package='scipy')
        tsim, profiles = sim.simulate(
                numpoints=100, integrator='vode'
            )
        sim.initialize_model()
        """

        # Discretize the model using finite difference method
        discretizer = TransformationFactory("dae.finite_difference")
        discretizer.apply_to(m, nfe=len(time) - 1, scheme="BACKWARD", wrt=m.t)

        # discretizer = TransformationFactory('dae.collocation')
        # discretizer.apply_to(m, nfe=len(time)-1, ncp=3, wrt=m.t)

        # Fix the input values
        for i, t in enumerate(self.data["t"]):
            m.u[t].fix(self.data["u"].iloc[i])

        return None

    def label_experiment(self):
        """
        In this method, we label the model with the experimental data. We need to have the `experiment_inputs`, `experiment_outputs`, `measurement_error`
        and `unknown_parameters` in the model
        """
        m = self.model

        time = self.data["t"].to_numpy()
        x1_data = self.data["x1"].to_numpy()
        x2_data = self.data["x2"].to_numpy()

        # print("len(m.t): ", len(m.t))
        # print("len(x1_data): ", len(x1_data))

        # Set measurement labels
        m.experiment_outputs = Suffix(direction=Suffix.LOCAL)

        # Add x1 measurement outputs
        m.experiment_outputs.update((m.x1[t], x1_data[i]) for i, t in enumerate(time))

        # Add x2 measurement outputs
        m.experiment_outputs.update((m.x2[t], x2_data[i]) for i, t in enumerate(time))

        # Add measurement error
        measurement_error = 0.1
        m.measurement_error = Suffix(direction=Suffix.LOCAL)
        m.measurement_error.update((m.x1[t], measurement_error) for t in m.t)
        m.measurement_error.update((m.x2[t], measurement_error) for t in m.t)

        # Specify experiment design variables (inputs)
        m.experiment_inputs = Suffix(direction=Suffix.LOCAL)
        m.experiment_inputs.update((m.u[t], None) for t in m.t)

        # Add unknown parameter labels
        m.unknown_parameters = Suffix(direction=Suffix.LOCAL)
        m.unknown_parameters.update((k, value(k)) for k in [m.a11, m.a22, m.b1])

        return None

    def get_labeled_model(self):
        """We absolutely need the `get_labeled_model()` method to return the model with the labels. Otherwise, the `parmest` and `DOE` will not work."""
        if self.model is None:
            self.create_model()
            self.finalize_model()
            self.label_experiment()
        return self.model